In [3]:
import requests
from bs4 import BeautifulSoup
import json

# 1) Neun Bundesländer-Kürzel
bundeslaender = [
    "burgenland",
    "kaernten",
    "niederoesterreich",
    "oberoesterreich",
    "salzburg",
    "steiermark",
    "tirol",
    "vorarlberg",
    "wien"
]

alle_warnungen = []

for land in bundeslaender:
    url = f"http://www.wetterwarnungen.at/html/de/heute/gewitter/at/{land}/"
    try:
        resp = requests.get(url, timeout=30)
        resp.raise_for_status()
    except requests.RequestException as e:
        print(f"Fehler beim Abrufen von {url}: {e}")
        continue

    soup = BeautifulSoup(resp.text, "html.parser")

    # 2) Bundesland-Namen ermitteln
    # In der Regel steht der Name als <h1> oder <h2> oder <h3> oberhalb der Tabelle
    h1 = soup.find("h1")
    if h1 and h1.text.strip():
        bundesland_name = h1.text.strip()
    else:
        # Fallback: Kürzel → Lesbare Form ("niederoesterreich" → "Niederösterreich")
        bundesland_name = land.capitalize() \
            .replace("niederoesterreich", "Niederösterreich") \
            .replace("oberoesterreich", "Oberösterreich") \
            .replace("kaernten", "Kärnten") \
            .replace("steiermark", "Steiermark") \
            .replace("vorarlberg", "Vorarlberg")

    # 3) Warn-Liste-Tabelle finden
    # Aktueller CSS-Selektor (prüfe mit F12, ob Klassenname noch 'warnlist' ist)
    table = soup.find("table", class_="warnlist")
    if not table:
        # Falls keine Tabelle gefunden: Hinweis ausgeben und weiter zum nächsten Bundesland
        print(f"Keine Gewitterwarn-Tabelle gefunden für {land}")
        continue

    # 4) Jede Zeile (<tr>) im <tbody> durchlaufen
    for row in table.select("tbody tr"):
        cells = row.find_all("td")
        if len(cells) < 5:
            # Manche Zeilen können z. B. Leerzeilen enthalten—diese überspringen
            continue

        warnstufe    = cells[0].get_text(strip=True)
        ereignis     = cells[1].get_text(strip=True)
        von           = cells[2].get_text(strip=True)  # "Von"
        bis           = cells[3].get_text(strip=True)  # "Bis"
        beschreibung = cells[4].get_text(strip=True)

        warnung = {
            "country":      "AT",                # Österreich
            "region":       bundesland_name,     # z. B. "Oberösterreich"
            "warnstufe":    warnstufe,           # "Rot", "Orange", "Gelb"
            "ereignis":     ereignis,            # z. B. "Gewitter"
            "start_time":   von,                 # "2025-06-01 14:00"
            "end_time":     bis,                 # "2025-06-01 20:00"
            "beschreibung": beschreibung         # z. B. "Orkanböen bis 110 km/h"
        }

        alle_warnungen.append(warnung)

# 5) Alle gesammelten Warnungen ausgeben (oder in Datei speichern)
for w in alle_warnungen:
    print(json.dumps(w, ensure_ascii=False))

# Oder in eine JSON-Datei schreiben:
# with open("zamg_warnungen_at.json", "w", encoding="utf-8") as f:
#     json.dump(alle_warnungen, f, ensure_ascii=False, indent=2)


Keine Gewitterwarn-Tabelle gefunden für burgenland
Keine Gewitterwarn-Tabelle gefunden für kaernten
Keine Gewitterwarn-Tabelle gefunden für niederoesterreich
Keine Gewitterwarn-Tabelle gefunden für oberoesterreich
Keine Gewitterwarn-Tabelle gefunden für salzburg
Keine Gewitterwarn-Tabelle gefunden für steiermark
Keine Gewitterwarn-Tabelle gefunden für tirol
Keine Gewitterwarn-Tabelle gefunden für vorarlberg
Keine Gewitterwarn-Tabelle gefunden für wien
